# Create NA Plants Non-Fixed Voucher

## Input:

- Path to Excel file containing claims with TPLs
- start and end voucher dates
- honda/acura hourly labor rates
- part cost factor
- handling cost factor

## Output:

- Excel voucher containing NA Plants' non-fixed voucher

In [8]:
from datetime import datetime
from pathlib import Path
from win10toast import ToastNotifier
import numpy as np
import pandas as pd
import os
import os.path as op
import pyodbc
import scrapbook as sb
pd.options.display.max_columns=100
pd.options.display.max_rows=1000
pd.options.display.float_format = '{:20,.2f}'.format

### Parameters needed to generate voucher:

In [84]:
start_voucher_date = '2018-09-01'
end_voucher_date = '2018-09-30'
honda_hourly_labor_rate = 120
acura_hourly_labor_rate = 141
part_cost_factor = 0.9
handling_cost_factor = 0.63

### Get current working directory:

In [85]:
cwd = os.getcwd()
cwd

'D:\\jupyter\\projects\\ZF_Automation\\papermill\\prod'

In [87]:
nb = sb.read_notebook(cwd + '\\output\\CLA_Claims_With_or_Without_TPLs.ipynb')

# Run this instead for testing purposes outside of using papermill
# nb = sb.read_notebook(cwd + '\\1_Obtain_CLA_Claims_With_or_Without_TPLs.ipynb')

In [88]:
nb.scraps['path_to_claims_with_tpl'].data

'\\\\mmpapp02\\mq_db\\wrp\\ZF_ATM_WRP_Project\\automated_vouchers\\outputs\\excel\\NorthAmericanPlants\\NonFixed\\2019-04-23\\CLA_Claims_With_TPLs.xlsx'

In [89]:
claims_with_tpl = pd.read_excel(nb.scraps['path_to_claims_with_tpl'].data, index=False)

In [90]:
claims_with_tpl.head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM
0,5FRYD4H85HB000032251369312271,201612,5FRYD4H85HB000032,5FRYD4H85HB000032,2016-12-23,2017-01-27,NaT,NaT,2017-01-27,CLOSED,CLOSED,6200,TRANSMISSION,35.00,35,062015NCA31,"CUSTOMER, STATES VEHICLE WILL NOT GO IN PARK A...",Visual inspection revealed ferrous debris in H...,Transmission shifted to Park after ignition wa...,HCU Contamination,HCU contamination caused claim of transmission...,251369,312271,"CUSTOMER, STATES VEHICLE WILL NOT GO IN PARK A...",2016-12-10,2016-12-19,2016-06-07,2016-06-06,218102,101,15A,KA,5.90,150.00,2.20,2016-12-21 10:28:30.350,100.00,Supplier Mfg. Defect,5267,$,"2,002.35","1,041.00","1,109.00","4,152.35",N,3214,Normal,1,6200,062005NCA31RM,WARRANTY A/T KIT,1,HTZ6,AJ6,,Q5NC0147177,ACURA,2017,HMA,MDX,9,200215NC,8.10,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",710960,1
1,5FRYD4H88HB000042251406314429,201803,5FRYD4H88HB000042,5FRYD4H88HB000042,2018-04-24,2018-06-07,2018-06-07,2018-06-26,2018-06-26,CLOSED,CLOSED,6200,TRANSMISSION,63.00,63,062275J4010,"ATF WARMER LEAKED AND, CAUSED TRANSMISSION TO ...",~ Defective oil cooler\r\n,~,Oil Cooler Diaphram Damaged,NaN,251406,314429,"ATF WARMER LEAKED AND, CAUSED TRANSMISSION TO ...",2018-03-16,2018-03-23,2016-06-07,2016-06-06,218102,101,15A,KA,5.90,0.00,2.20,2018-04-20 10:34:11.400,100.00,Supplier Mfg. Defect,18360,$,"2,891.96","1,015.99","2,445.23","6,353.18",N,8103,Normal,1,6200,062005NCA31RM ...,WARRANTY A/T KIT,1,HTZ6,AJ6,,Q5NC0147152,ACURA,2017,HMA,MDX,9,200215NC,8.10,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",737496,1
2,5FRYD4H52HB000091251061931424,201705,5FRYD4H52HB000091,5FRYD4H52HB000091,2017-06-16,2017-08-07,2017-08-08,2017-09-20,2017-09-20,CLOSED,CLOSED,6200,TRANSMISSION,96.00,96,062005NCA21,CUSTOMER STATES TRANSMISSION IS SLIPPING,~ Completed visual inspection of transmission...,"~ During teardown drained oil, and found milk...",Oil Cooler Diaphram Damaged,Known issue related to oil cooler.,251061,931424,CUSTOMER STATES TRANSMISSION IS SLIPPING,2017-04-21,2017-05-09,2016-06-14,2016-06-13,218102,101,15A,KA,5.90,125.00,2.50,2017-06-14 11:12:08.720,100.00,Supplier Mfg. Defect,5533,$,"3,024.98","1,259.00","2,223.01","6,506.99",N,2301,Normal,1,6200,062005NCA21RM,WARRANTY A/T KIT,1,HTZ6,AG6,,Q5NC0147171,ACURA,2017,HMA,MDX,9,200215NC,8.40,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",722128,1
3,5FRYD4H52HB000091251061953135,201706,5FRYD4H52HB000091,5FRYD4H52HB000091,2017-06-16,2017-08-07,2017-08-09,2017-09-21,2017-09-21,CLOSED,CLOSED,6200,TRANSMISSION,97.00,97,062005NCA21,CUSTOMER STATES VEHICLE DOES NOT HAVE POWER SI...,~ Completed visual inspection of transmission.,"~ Drained oil, and found milky oil. Removed o...",Oil Cooler Diaphram Damaged,Known issue related to oil cooler,251061,953135,CUSTOMER STATES VEHICLE

### Add WRPID column which is in the format: WRP219B + 2-digit month + 2-digit year where month/year depends on end voucher date

In [91]:
year = end_voucher_date[2:4]
year

'18'

In [92]:
month = end_voucher_date[5:7]
month

'09'

In [93]:
claims_with_tpl = claims_with_tpl.assign(WRPID='WRP219B' + month + year)

#### Let's confirm that ```WRPID``` has been created (scroll to far right):

In [94]:
claims_with_tpl.head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,WRPID
0,5FRYD4H85HB000032251369312271,201612,5FRYD4H85HB000032,5FRYD4H85HB000032,2016-12-23,2017-01-27,NaT,NaT,2017-01-27,CLOSED,CLOSED,6200,TRANSMISSION,35.00,35,062015NCA31,"CUSTOMER, STATES VEHICLE WILL NOT GO IN PARK A...",Visual inspection revealed ferrous debris in H...,Transmission shifted to Park after ignition wa...,HCU Contamination,HCU contamination caused claim of transmission...,251369,312271,"CUSTOMER, STATES VEHICLE WILL NOT GO IN PARK A...",2016-12-10,2016-12-19,2016-06-07,2016-06-06,218102,101,15A,KA,5.90,150.00,2.20,2016-12-21 10:28:30.350,100.00,Supplier Mfg. Defect,5267,$,"2,002.35","1,041.00","1,109.00","4,152.35",N,3214,Normal,1,6200,062005NCA31RM,WARRANTY A/T KIT,1,HTZ6,AJ6,,Q5NC0147177,ACURA,2017,HMA,MDX,9,200215NC,8.10,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",710960,1,WRP219B0918
1,5FRYD4H88HB000042251406314429,201803,5FRYD4H88HB000042,5FRYD4H88HB000042,2018-04-24,2018-06-07,2018-06-07,2018-06-26,2018-06-26,CLOSED,CLOSED,6200,TRANSMISSION,63.00,63,062275J4010,"ATF WARMER LEAKED AND, CAUSED TRANSMISSION TO ...",~ Defective oil cooler\r\n,~,Oil Cooler Diaphram Damaged,NaN,251406,314429,"ATF WARMER LEAKED AND, CAUSED TRANSMISSION TO ...",2018-03-16,2018-03-23,2016-06-07,2016-06-06,218102,101,15A,KA,5.90,0.00,2.20,2018-04-20 10:34:11.400,100.00,Supplier Mfg. Defect,18360,$,"2,891.96","1,015.99","2,445.23","6,353.18",N,8103,Normal,1,6200,062005NCA31RM ...,WARRANTY A/T KIT,1,HTZ6,AJ6,,Q5NC0147152,ACURA,2017,HMA,MDX,9,200215NC,8.10,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",737496,1,WRP219B0918
2,5FRYD4H52HB000091251061931424,201705,5FRYD4H52HB000091,5FRYD4H52HB000091,2017-06-16,2017-08-07,2017-08-08,2017-09-20,2017-09-20,CLOSED,CLOSED,6200,TRANSMISSION,96.00,96,062005NCA21,CUSTOMER STATES TRANSMISSION IS SLIPPING,~ Completed visual inspection of transmission...,"~ During teardown drained oil, and found milk...",Oil Cooler Diaphram Damaged,Known issue related to oil cooler.,251061,931424,CUSTOMER STATES TRANSMISSION IS SLIPPING,2017-04-21,2017-05-09,2016-06-14,2016-06-13,218102,101,15A,KA,5.90,125.00,2.50,2017-06-14 11:12:08.720,100.00,Supplier Mfg. Defect,5533,$,"3,024.98","1,259.00","2,223.01","6,506.99",N,2301,Normal,1,6200,062005NCA21RM,WARRANTY A/T KIT,1,HTZ6,AG6,,Q5NC0147171,ACURA,2017,HMA,MDX,9,200215NC,8.40,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",722128,1,WRP219B0918
3,5FRYD4H52HB000091251061953135,201706,5FRYD4H52HB000091,5FRYD4H52HB000091,2017-06-16,2017-08-07,2017-08-09,2017-09-21,2017-09-21,CLOSED,CLOSED,6200,TRANSMISSION,97.00,97,062005NCA21,CUSTOMER STATES VEHICLE DOES NOT HAVE POWER SI...,~ Completed visual inspection of transmission.,"~ Drained oil, and found milky oil. Removed o...",Oil Cooler Diaphram Damaged,Known issue related to oil co

### Add ```HRLY_RATE_USD``` column:

In [95]:
claims_with_tpl['HRLY_RATE_USD'] = np.where(claims_with_tpl['NAMEPLATE'] == 'ACURA', int(acura_hourly_labor_rate), int(honda_hourly_labor_rate))

#### Let's confirm that ```HRLY_RATE_USD``` column has been created (scroll to far right):

In [96]:
claims_with_tpl.query("NAMEPLATE == 'ACURA'").head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,WRPID,HRLY_RATE_USD
0,5FRYD4H85HB000032251369312271,201612,5FRYD4H85HB000032,5FRYD4H85HB000032,2016-12-23,2017-01-27,NaT,NaT,2017-01-27,CLOSED,CLOSED,6200,TRANSMISSION,35.00,35,062015NCA31,"CUSTOMER, STATES VEHICLE WILL NOT GO IN PARK A...",Visual inspection revealed ferrous debris in H...,Transmission shifted to Park after ignition wa...,HCU Contamination,HCU contamination caused claim of transmission...,251369,312271,"CUSTOMER, STATES VEHICLE WILL NOT GO IN PARK A...",2016-12-10,2016-12-19,2016-06-07,2016-06-06,218102,101,15A,KA,5.90,150.00,2.20,2016-12-21 10:28:30.350,100.00,Supplier Mfg. Defect,5267,$,"2,002.35","1,041.00","1,109.00","4,152.35",N,3214,Normal,1,6200,062005NCA31RM,WARRANTY A/T KIT,1,HTZ6,AJ6,,Q5NC0147177,ACURA,2017,HMA,MDX,9,200215NC,8.10,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",710960,1,WRP219B0918,141
1,5FRYD4H88HB000042251406314429,201803,5FRYD4H88HB000042,5FRYD4H88HB000042,2018-04-24,2018-06-07,2018-06-07,2018-06-26,2018-06-26,CLOSED,CLOSED,6200,TRANSMISSION,63.00,63,062275J4010,"ATF WARMER LEAKED AND, CAUSED TRANSMISSION TO ...",~ Defective oil cooler\r\n,~,Oil Cooler Diaphram Damaged,NaN,251406,314429,"ATF WARMER LEAKED AND, CAUSED TRANSMISSION TO ...",2018-03-16,2018-03-23,2016-06-07,2016-06-06,218102,101,15A,KA,5.90,0.00,2.20,2018-04-20 10:34:11.400,100.00,Supplier Mfg. Defect,18360,$,"2,891.96","1,015.99","2,445.23","6,353.18",N,8103,Normal,1,6200,062005NCA31RM ...,WARRANTY A/T KIT,1,HTZ6,AJ6,,Q5NC0147152,ACURA,2017,HMA,MDX,9,200215NC,8.10,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",737496,1,WRP219B0918,141
2,5FRYD4H52HB000091251061931424,201705,5FRYD4H52HB000091,5FRYD4H52HB000091,2017-06-16,2017-08-07,2017-08-08,2017-09-20,2017-09-20,CLOSED,CLOSED,6200,TRANSMISSION,96.00,96,062005NCA21,CUSTOMER STATES TRANSMISSION IS SLIPPING,~ Completed visual inspection of transmission...,"~ During teardown drained oil, and found milk...",Oil Cooler Diaphram Damaged,Known issue related to oil cooler.,251061,931424,CUSTOMER STATES TRANSMISSION IS SLIPPING,2017-04-21,2017-05-09,2016-06-14,2016-06-13,218102,101,15A,KA,5.90,125.00,2.50,2017-06-14 11:12:08.720,100.00,Supplier Mfg. Defect,5533,$,"3,024.98","1,259.00","2,223.01","6,506.99",N,2301,Normal,1,6200,062005NCA21RM,WARRANTY A/T KIT,1,HTZ6,AG6,,Q5NC0147171,ACURA,2017,HMA,MDX,9,200215NC,8.40,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",722128,1,WRP219B0918,141
3,5FRYD4H52HB000091251061953135,201706,5FRYD4H52HB000091,5FRYD4H52HB000091,2017-06-16,2017-08-07,2017-08-09,2017-09-21,2017-09-21,CLOSED,CLOSED,6200,TRANSMISSION,97.00,97,062005NCA21,CUSTOMER STATES VEHICLE DOES NOT HAVE POWER SI...,~ Completed visual inspection of transmission.,"~ Drained oil, and found milky oil. Removed o...",Oil Cooler Diaphram Damaged,Kno

In [97]:
claims_with_tpl.query("NAMEPLATE == 'HONDA'").head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,WRPID,HRLY_RATE_USD
515,5FNRL6H77JB002639208424805428,201812,5FNRL6H77JB002639,5FNRL6H77JB002639,2019-01-22,2019-03-06,2019-03-06,2019-03-11,2019-03-11,AVAILABLE_FOR_REVIEW,CLOSED,6280,KIT,48.00,48,062805NZA31,CUSTOMER STATES VEHICLE SHUTTERS WHEN SHIFTING,~,~,No Analysis,NaN,208424,805428,Customer States vehicle shutters when shifting,2018-12-10,2018-12-21,2017-04-26,2017-04-25,223199,101,15A,KA,0.60,0.00,0.60,2019-01-18 14:07:00.447,0.00,Systems Related,21036,$,282.57,159.60,173.89,616.06,N,3214,Normal,1,6280,062805NZA31 ...,"CONTROL UNIT, TRANS",1,JTHR,AJ6,,Q5NZ0183538,HONDA,2018,HMA,ODYSSEY,9,200215NZ,1.20,1.00,443.86,138.81,399.47,277.62,277.62,748155,1,WRP219B0918,120
516,5FNRL6H76JB003023206648752829,201712,5FNRL6H76JB003023,5FNRL6H76JB003023,2018-01-04,2018-02-16,2019-02-06,2019-03-14,2019-03-14,AVAILABLE_FOR_REVIEW,CLOSED,6200,TRANSMISSION,434.00,434,062015NZA11,C/S THE VEHICLE IS SHIFTING ABNORMALLY. WHEN D...,~ No visual defects found.,~ Full teardown performed with no findings. A...,Dealer Error,Dealer's Fault: Oil Level Adjustment Not OK.\r...,206648,752829,C/S THE VEHICLE IS SHIFTING ABNORMALLY. WHEN D...,2017-11-06,2017-11-30,2017-04-27,2017-04-26,218102,101,15A,KA,6.50,125.00,0.70,2018-01-02 07:50:59.447,0.00,Dealer Error,5379,$,"2,731.52",953.00,"1,939.38","5,623.90",N,504,Normal,1,6200,062005NZA11RM ...,WARRANTY A/T KIT,1,JTHR,AG8,,Q5NZ0184162,HONDA,2018,HMA,ODYSSEY,9,200215NZ,7.20,1.00,"2,871.34","2,445.31","2,584.21","4,890.62","2,584.21",732376,1,WRP219B0918,120
517,5FNRL6H73JB003299206500724651,201805,5FNRL6H73JB003299,5FNRL6H73JB003299,2018-06-26,2018-08-15,2019-03-25,NaT,NaT,REOPENED_FOR_ANALYSIS,OPEN,6200,TRANSMISSION,nan,301,062005NZA31,CUSTOMER STATES ROAD DEPARTURE MITIGATION ; AC...,~,~,NaN,NaN,206500,724651,CUSTOMER STATES ROAD DEPARTURE MITIGATION ; AC...,2018-05-10,2018-05-16,2017-04-28,2017-04-27,218102,101,15A,KA,6.50,157.50,0.70,2018-06-22 08:28:55.577,nan,NaN,43516,$,"2,847.09","1,158.30","2,567.21","6,572.60",N,504,Normal,1,6200,062005NZA31RM ...,WARRANTY A/T KIT,1,JTHR,AG8,,Q5NZ0184760,HONDA,2018,HMA,ODYSSEY,9,200215NZ,7.20,1.00,"2,871.34","2,337.99","2,584.21","4,675.98","2,584.21",740189,1,WRP219B0918,120
518,5FNRL6H78JB003346208703021248,201803,5FNRL6H78JB003346,5FNRL6H78JB003346,2018-05-21,2018-07-11,2019-02-06,2019-03-11,2019-03-11,AVAILABLE_FOR_REVIEW,CLOSED,6280,KIT,294.00,294,062805NZA11,"C/S: CUSTOMER STATES, HEARS A CLUNK FROM THE T...",~,~,No Analysis,GBL data only shows engine intervention fault ...,208703,021248,"C/S: CUSTOMER STATES, HEARS A CLUNK FROM THE T...",2018-03-05,2018-03-06,2017-05-01,2017-04-28,2231A0,101,15A,KA,0.60,0.00,0.00,2018-05-17 14:14:43.370,0.00,Systems Related,15541,$,282.57,78.00,173.89,534.46,N,3217,Normal,1,6280,062805NZA11 ...,"CONTROL UNIT, TRANS",1,J

### Add calculated costs:

#### Minimum part cost (lesser of: 2 x FOB or 90% dealer net) was already defined in the main SQL query:

In [98]:
claims_with_tpl = claims_with_tpl.assign(CALCULATED_PART_COST_USD=claims_with_tpl['MIN_PART_COST_USD'])

#### Calculated flat rate labor cost (USD) is just the FLAT_RATE_HRS_QTY x nameplate hourly labor rate:

In [99]:
claims_with_tpl['CALCULATED_FLAT_RATE_LABOR_COST_USD'] = np.where(claims_with_tpl['NAMEPLATE'] == 'ACURA',
                                                        claims_with_tpl['FLAT_RATE_HRS_QTY'] * int(acura_hourly_labor_rate),
                                                        claims_with_tpl['FLAT_RATE_HRS_QTY'] * int(honda_hourly_labor_rate))

#### Calculated diagnostic labor cost (USD) = (claim labor hours x hourly labor rate) - (flat rate hours x hourly labor rate)

In [100]:
claims_with_tpl = claims_with_tpl.assign(CALCULATED_DIAGNOSTIC_LABOR_COST_USD=(claims_with_tpl['CLAIM_LABOR_HRS_QTY'] *
                               claims_with_tpl['HRLY_RATE_USD']) - (claims_with_tpl['FLAT_RATE_HRS_QTY'] *
                               claims_with_tpl['HRLY_RATE_USD']))

#### Calculated handling cost = dealer net part cost x handling cost factor:

In [101]:
claims_with_tpl = claims_with_tpl.assign(CALCULATED_HDLG_CHG_AMT_USD=claims_with_tpl['DLR_NET_PRCE_AMT_USD'] *
                               float(handling_cost_factor))

#### Calculated total charge = part cost + labor cost + handling cost + sublet/freight/tax (CLA labor - WHA labor)

In [102]:
claims_with_tpl['CALCULATED_TOTAL_COST_USD'] = np.where(claims_with_tpl['ACTUAL_LABOR_CHG_AMT_USD'] == 0,
            claims_with_tpl['CALCULATED_PART_COST_USD'] + claims_with_tpl['CALCULATED_HDLG_CHG_AMT_USD'],
            claims_with_tpl['CALCULATED_PART_COST_USD'] + claims_with_tpl['CALCULATED_FLAT_RATE_LABOR_COST_USD'] +
            claims_with_tpl['CALCULATED_DIAGNOSTIC_LABOR_COST_USD'] + claims_with_tpl['CALCULATED_HDLG_CHG_AMT_USD'] +
                                                       claims_with_tpl['FRGT_SUBL_TAX_AMT_USD']
        )

#### Calculated total labor cost (USD) = calculated flat rate labor cost + calculated diagnostic labor cost:

In [103]:
claims_with_tpl = claims_with_tpl.assign(CALCULATED_TOTAL_LABOR_COST_USD=claims_with_tpl['CALCULATED_FLAT_RATE_LABOR_COST_USD'] +
                               claims_with_tpl['CALCULATED_DIAGNOSTIC_LABOR_COST_USD'])

### Add ```ROW_NUM``` column to identify duplicate records due to claim adjustments:

In [104]:
claims_with_tpl['ROW_NUM'] = claims_with_tpl.sort_values(by=['RECORD_DESC'], ascending=False) \
                                               .groupby(['VIN_DLR_CLM_SK']).cumcount() + 1

In [105]:
claims_with_tpl.head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,WRPID,HRLY_RATE_USD,CALCULATED_PART_COST_USD,CALCULATED_FLAT_RATE_LABOR_COST_USD,CALCULATED_DIAGNOSTIC_LABOR_COST_USD,CALCULATED_HDLG_CHG_AMT_USD,CALCULATED_TOTAL_COST_USD,CALCULATED_TOTAL_LABOR_COST_USD,ROW_NUM
0,5FRYD4H85HB000032251369312271,201612,5FRYD4H85HB000032,5FRYD4H85HB000032,2016-12-23,2017-01-27,NaT,NaT,2017-01-27,CLOSED,CLOSED,6200,TRANSMISSION,35.00,35,062015NCA31,"CUSTOMER, STATES VEHICLE WILL NOT GO IN PARK A...",Visual inspection revealed ferrous debris in H...,Transmission shifted to Park after ignition wa...,HCU Contamination,HCU contamination caused claim of transmission...,251369,312271,"CUSTOMER, STATES VEHICLE WILL NOT GO IN PARK A...",2016-12-10,2016-12-19,2016-06-07,2016-06-06,218102,101,15A,KA,5.90,150.00,2.20,2016-12-21 10:28:30.350,100.00,Supplier Mfg. Defect,5267,$,"2,002.35","1,041.00","1,109.00","4,152.35",N,3214,Normal,1,6200,062005NCA31RM,WARRANTY A/T KIT,1,HTZ6,AJ6,,Q5NC0147177,ACURA,2017,HMA,MDX,9,200215NC,8.10,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",710960,1,WRP219B0918,141,"2,815.14",831.90,310.20,"1,970.60","6,077.84","1,142.10",1
1,5FRYD4H88HB000042251406314429,201803,5FRYD4H88HB000042,5FRYD4H88HB000042,2018-04-24,2018-06-07,2018-06-07,2018-06-26,2018-06-26,CLOSED,CLOSED,6200,TRANSMISSION,63.00,63,062275J4010,"ATF WARMER LEAKED AND, CAUSED TRANSMISSION TO ...",~ Defective oil cooler\r\n,~,Oil Cooler Diaphram Damaged,NaN,251406,314429,"ATF WARMER LEAKED AND, CAUSED TRANSMISSION TO ...",2018-03-16,2018-03-23,2016-06-07,2016-06-06,218102,101,15A,KA,5.90,0.00,2.20,2018-04-20 10:34:11.400,100.00,Supplier Mfg. Defect,18360,$,"2,891.96","1,015.99","2,445.23","6,353.18",N,8103,Normal,1,6200,062005NCA31RM ...,WARRANTY A/T KIT,1,HTZ6,AJ6,,Q5NC0147152,ACURA,2017,HMA,MDX,9,200215NC,8.10,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",737496,1,WRP219B0918,141,"2,815.14",831.90,310.20,"1,970.60","5,927.84","1,142.10",1
2,5FRYD4H52HB000091251061931424,201705,5FRYD4H52HB000091,5FRYD4H52HB000091,2017-06-16,2017-08-07,2017-08-08,2017-09-20,2017-09-20,CLOSED,CLOSED,6200,TRANSMISSION,96.00,96,062005NCA21,CUSTOMER STATES TRANSMISSION IS SLIPPING,~ Completed visual inspection of transmission...,"~ During teardown drained oil, and found milk...",Oil Cooler Diaphram Damaged,Known issue related to oil cooler.,251061,931424,CUSTOMER STATES TRANSMISSION IS SLIPPING,2017-04-21,2017-05-09,2016-06-14,2016-06-13,218102,101,15A,KA,5.90,125.00,2.50,2017-06-14 11:12:08.720,100.00,Supplier Mfg. Defect,5533,$,"3,024.98","1,259.00","2,223.01","6,506.99",N,2301,Normal,1,6200,062005NCA21RM,WARRANTY A/T KIT,1,HTZ6,AG6,,Q5NC0147171,ACURA,2017,HMA,MDX,9,200215NC,8.40,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",722128,1,WRP219B0918,141,"2,815.14",831.90,352.50,"1,970.60","6,095.14","1,

In [106]:
duplicate_rows = claims_with_tpl[claims_with_tpl.duplicated(subset=['VIN_DLR_CLM_SK'], keep=False)]
duplicate_rows.sort_values(['VIN_DLR_CLM_SK', 'ROW_NUM'])[:5]

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,WRPID,HRLY_RATE_USD,CALCULATED_PART_COST_USD,CALCULATED_FLAT_RATE_LABOR_COST_USD,CALCULATED_DIAGNOSTIC_LABOR_COST_USD,CALCULATED_HDLG_CHG_AMT_USD,CALCULATED_TOTAL_COST_USD,CALCULATED_TOTAL_LABOR_COST_USD,ROW_NUM
1359,19UUB2F52HA003984251551361711,201807,19UUB2F52HA003984,19UUB2F52HA003984,2018-07-17,2018-08-29,2018-09-06,2018-09-07,2018-09-07,CLOSED,CLOSED,6200,TRANSMISSION,52.00,52,062015J4A73,CUSTOMER STATES TRANSMISSION SYSTEM PROBLEM LI...,~ Visual Inspection of transmission revealed ...,~ Full teardown performed and all clutches me...,NTF,HCU and sensor unit were sent for functional t...,251551,361711,CUSTOMER STATES TRANSMISSION SYSTEM PROBLEM LI...,2018-06-26,2018-07-05,2016-12-07,2016-12-06,218102,101,101,KA,5.80,125.00,1.40,2018-07-13 08:27:44.780,0.00,NTF,24298,$,"2,769.59","1,025.00","1,107.84","4,902.43",N,3214,Normal,1,6200,062005J4A71RM ...,WARRANTY A/T KIT,1,HTZ3,AB9,,Q5J40154391,ACURA,2017,MAP,TLX,9,200215J4,7.20,1.00,"2,967.12","2,433.04","2,670.41","4,866.08","2,670.41",740894,1,WRP219B0918,141,"2,670.41",817.80,197.40,"1,869.29","5,679.90","1,015.20",1
1358,19UUB2F52HA003984251551361711,201902,19UUB2F52HA003984,19UUB2F52HA003984,2018-07-17,2018-08-29,2018-09-06,2018-09-07,2018-09-07,CLOSED,CLOSED,6200,TRANSMISSION,52.00,52,062015J4A73,CUSTOMER STATES TRANSMISSION SYSTEM PROBLEM LI...,~ Visual Inspection of transmission revealed ...,~ Full teardown performed and all clutches me...,NTF,HCU and sensor unit were sent for functional t...,251551,361711,CUSTOMER STATES TRANSMISSION SYSTEM PROBLEM LI...,2018-06-26,2018-07-05,2016-12-07,2016-12-06,218102,101,101,KA,5.80,125.00,-5.80,2018-07-13 08:27:44.780,0.00,NTF,24298,$,"-1,800.23","-1,025.00","-1,107.84","-3,933.07",N,99900,Debit,-1,6200,062005J4A71RM ...,WARRANTY A/T KIT,1,HTZ3,AB9,,Q5J40154391,ACURA,2017,MAP,TLX,9,200215J4,0.00,1.00,"2,967.12","2,433.04","2,670.41","4,866.08","2,670.41",740894,1,WRP219B0918,141,"2,670.41",817.80,-817.80,"1,869.29","4,664.70",0.00,2
1303,19UUB2F57HA001020251024381631,201811,19UUB2F57HA001020,19UUB2F57HA001020,2018-12-17,2019-02-07,2019-03-25,NaT,NaT,REOPENED_FOR_ANALYSIS,OPEN,6200,TRANSMISSION,nan,127,062005J4A71RM,MISC REPAIR CUSTOMER STATES THAT TRANSMISSION ...,~,~,NaN,NaN,251024,381631,MISC REPAIR CUSTOMER STATES THAT TRANSMISSION ...,2018-10-16,2018-11-16,2016-08-26,2016-08-24,218102,101,101,KA,5.80,250.00,1.40,2018-12-13 09:49:05.587,nan,NaN,30963,$,"2,710.89","1,257.64","1,084.35","5,052.88",N,3214,Normal,1,6200,062005J4A71RM ...,WARRANTY A/T KIT,1,HTZ3,AB9,,Q5J40136178,ACURA,2017,MAP,TLX,9,200215J4,7.20,1.00,"2,967.12","2,433.04","2,670.41","4,866.08","2,670.41",746531,1,WRP219B0918,141,"2,670.41",817.80,197.40,"1,869.29","5,804.90","1,015.20",1
1306,19UUB2F57HA001020251024381631,201902,19UUB2F57HA001020,19UUB2F57HA001020,

### Add actual claim level total cost:

In [107]:
pivot = pd.pivot_table(claims_with_tpl, index='VIN_DLR_CLM_SK', values='ACTUAL_TOTAL_CHG_AMT_USD', aggfunc='sum')

In [108]:
pivot.head()

,ACTUAL_TOTAL_CHG_AMT_USD
VIN_DLR_CLM_SK,
19UUB2F30HA001181251073449021,291.53
19UUB2F30HA001553251540110959,"4,868.29"
19UUB2F30HA001553251540121681,"6,619.66"
19UUB2F30HA001553251540122143,"6,594.12"
19UUB2F30KA004587251503903061,"5,794.50"


In [109]:
pivot.columns = ['ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD']
pivot.reset_index(level=0, inplace=True)
pivot.head()

,VIN_DLR_CLM_SK,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD
0,19UUB2F30HA001181251073449021,291.53
1,19UUB2F30HA001553251540110959,"4,868.29"
2,19UUB2F30HA001553251540121681,"6,619.66"
3,19UUB2F30HA001553251540122143,"6,594.12"
4,19UUB2F30KA004587251503903061,"5,794.50"


In [110]:
claims_with_tpl = pd.merge(claims_with_tpl, pivot, how='left', left_on='VIN_DLR_CLM_SK', right_on='VIN_DLR_CLM_SK')

In [111]:
duplicate_rows = claims_with_tpl[claims_with_tpl.duplicated(subset=['VIN_DLR_CLM_SK'], keep=False)]
duplicate_rows.sort_values(['VIN_DLR_CLM_SK', 'ROW_NUM'])[['VIN_DLR_CLM_SK','ROW_NUM','ACTUAL_TOTAL_CHG_AMT_USD','ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD']][:10]

,VIN_DLR_CLM_SK,ROW_NUM,ACTUAL_TOTAL_CHG_AMT_USD,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD
1359,19UUB2F52HA003984251551361711,1,"4,902.43",969.36
1358,19UUB2F52HA003984251551361711,2,"-3,933.07",969.36
1303,19UUB2F57HA001020251024381631,1,"5,052.88",948.81
1306,19UUB2F57HA001020251024381631,2,"-4,104.07",948.81
1421,19UUB2F78JA007562251406355639,1,323.96,0.00
1422,19UUB2F78JA007562251406355639,2,-323.96,0.00
1469,19UUB3F42KA003181251570855161,1,442.18,0.00
1470,19UUB3F42KA003181251570855161,2,-442.18,0.00
1462,19UUB3F43KA000371251581593310,1,"6,369.20",0.00
1461,19UUB3F43KA000371251581593310,2,"-6,369.20",0.00


### Obtain ASN receive dates

In [112]:
username_m = os.environ['windowsuser']
password_m = os.environ['mainframepwd']

In [113]:
%%time
cnxn_string = 'DSN=DSSOGW01;UID=' + username_m + ';PWD=' + password_m

cnxn = pyodbc.connect(cnxn_string)
cursor = cnxn.cursor()

sql = """
SELECT
    RTRIM(TRANS_SERIAL_NO) AS TRANS_SERIAL_NO,
    ASN_RECEIVE_DATE

FROM
    WAR.WRTRN1
    
WHERE
    TRANS_SERIAL_NO LIKE 'Q%'
"""

try:
    print("Obtaining ASN records 20,000 rows at a time...")
    asn = pd.DataFrame()
    for chunk in pd.read_sql(sql, cnxn, index_col=None, parse_dates=['ASN_RECEIVE_DATE'], chunksize=20000):
        asn = pd.concat([asn, chunk])

    # Close connections
    cursor.close()
    cnxn.close()
except:
    print("Error connecting to ASN database server")
    cursor.close()
    cnxn.close()
print("Finished obtainig ASN records")

Obtaining ASN records 20,000 rows at a time...
Finished obtainig ASN records
Wall time: 53.8 s


#### LEFT JOIN the asn data to the claims with TPL data:

In [114]:
claims_with_tpl = pd.merge(claims_with_tpl, asn, how='left', left_on='TRANS_SERIAL_NO', right_on='TRANS_SERIAL_NO')

In [115]:
claims_with_tpl.head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,WRPID,HRLY_RATE_USD,CALCULATED_PART_COST_USD,CALCULATED_FLAT_RATE_LABOR_COST_USD,CALCULATED_DIAGNOSTIC_LABOR_COST_USD,CALCULATED_HDLG_CHG_AMT_USD,CALCULATED_TOTAL_COST_USD,CALCULATED_TOTAL_LABOR_COST_USD,ROW_NUM,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD,ASN_RECEIVE_DATE
0,5FRYD4H85HB000032251369312271,201612,5FRYD4H85HB000032,5FRYD4H85HB000032,2016-12-23,2017-01-27,NaT,NaT,2017-01-27,CLOSED,CLOSED,6200,TRANSMISSION,35.00,35,062015NCA31,"CUSTOMER, STATES VEHICLE WILL NOT GO IN PARK A...",Visual inspection revealed ferrous debris in H...,Transmission shifted to Park after ignition wa...,HCU Contamination,HCU contamination caused claim of transmission...,251369,312271,"CUSTOMER, STATES VEHICLE WILL NOT GO IN PARK A...",2016-12-10,2016-12-19,2016-06-07,2016-06-06,218102,101,15A,KA,5.90,150.00,2.20,2016-12-21 10:28:30.350,100.00,Supplier Mfg. Defect,5267,$,"2,002.35","1,041.00","1,109.00","4,152.35",N,3214,Normal,1,6200,062005NCA31RM,WARRANTY A/T KIT,1,HTZ6,AJ6,,Q5NC0147177,ACURA,2017,HMA,MDX,9,200215NC,8.10,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",710960,1,WRP219B0918,141,"2,815.14",831.90,310.20,"1,970.60","6,077.84","1,142.10",1,"4,152.35",2016-06-03
1,5FRYD4H88HB000042251406314429,201803,5FRYD4H88HB000042,5FRYD4H88HB000042,2018-04-24,2018-06-07,2018-06-07,2018-06-26,2018-06-26,CLOSED,CLOSED,6200,TRANSMISSION,63.00,63,062275J4010,"ATF WARMER LEAKED AND, CAUSED TRANSMISSION TO ...",~ Defective oil cooler\r\n,~,Oil Cooler Diaphram Damaged,NaN,251406,314429,"ATF WARMER LEAKED AND, CAUSED TRANSMISSION TO ...",2018-03-16,2018-03-23,2016-06-07,2016-06-06,218102,101,15A,KA,5.90,0.00,2.20,2018-04-20 10:34:11.400,100.00,Supplier Mfg. Defect,18360,$,"2,891.96","1,015.99","2,445.23","6,353.18",N,8103,Normal,1,6200,062005NCA31RM ...,WARRANTY A/T KIT,1,HTZ6,AJ6,,Q5NC0147152,ACURA,2017,HMA,MDX,9,200215NC,8.10,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",737496,1,WRP219B0918,141,"2,815.14",831.90,310.20,"1,970.60","5,927.84","1,142.10",1,"6,353.18",2016-06-03
2,5FRYD4H52HB000091251061931424,201705,5FRYD4H52HB000091,5FRYD4H52HB000091,2017-06-16,2017-08-07,2017-08-08,2017-09-20,2017-09-20,CLOSED,CLOSED,6200,TRANSMISSION,96.00,96,062005NCA21,CUSTOMER STATES TRANSMISSION IS SLIPPING,~ Completed visual inspection of transmission...,"~ During teardown drained oil, and found milk...",Oil Cooler Diaphram Damaged,Known issue related to oil cooler.,251061,931424,CUSTOMER STATES TRANSMISSION IS SLIPPING,2017-04-21,2017-05-09,2016-06-14,2016-06-13,218102,101,15A,KA,5.90,125.00,2.50,2017-06-14 11:12:08.720,100.00,Supplier Mfg. Defect,5533,$,"3,024.98","1,259.00","2,223.01","6,506.99",N,2301,Normal,1,6200,062005NCA21RM,WARRANTY A/T KIT,1,HTZ6,AG6,,Q5NC0147171,ACURA,2017,HMA,MDX,9,200215NC,8.40,1.00,"3,127.93","2,049.71","2,815.14"

### Number of rows with missing ASN_RECEIVE_DATE

In [116]:
# Number of claims with missing ASN_RECEIVE_DATE
claims_with_tpl[claims_with_tpl['ASN_RECEIVE_DATE'].isna()].shape[0]

13

### Fill missing ASN_RECEIVE_DATE with engine build date minus 4 days:

In [117]:
claims_with_tpl['ASN_RECEIVE_DATE'].fillna(value=claims_with_tpl['ENGINE_BUILD_DATE'] - pd.Timedelta('4 days'), inplace=True)

In [118]:
# Number of claims with missing ASN_RECEIVE_DATE
claims_with_tpl[claims_with_tpl['ASN_RECEIVE_DATE'].isna()].shape[0]

0

In [119]:
assert claims_with_tpl[claims_with_tpl['ASN_RECEIVE_DATE'].isna()].shape[0] == 0

### With ```ASN_RECEIVE_DATE``` fully populated, create ```TransDTF``` column = RO date minus ASN receive date:

In [120]:
claims_with_tpl = claims_with_tpl.assign(TransDTF=(claims_with_tpl['REPAIR_ORDER_DATE']
                                         - claims_with_tpl['ASN_RECEIVE_DATE']).apply(lambda x: x.days))

In [121]:
claims_with_tpl[['REPAIR_ORDER_DATE','ASN_RECEIVE_DATE','TransDTF']].head()

,REPAIR_ORDER_DATE,ASN_RECEIVE_DATE,TransDTF
0,2016-12-10,2016-06-03,190
1,2018-03-16,2016-06-03,651
2,2017-04-21,2016-06-13,312
3,2017-05-20,2016-06-13,341
4,2018-05-11,2016-06-13,697


### Obtain Mass Pro Cost from E-Quote tables:

In [122]:
print("Connecting to DSNOGW01 server")
cnxn_string = 'DSN=DSNOGW01;UID=' + username_m + ';PWD=' + password_m

cnxn = pyodbc.connect(cnxn_string)
cursor = cnxn.cursor()

sql = """
SELECT
    *

FROM (

SELECT
    EXREP1.NEW_STATUS,
    EXCOD1.COD_NAME,
    EXWRK1.L1_DSG_CHG_PRT_NO,
    LEFT(EXWRK1.L1_DSG_CHG_PRT_NO,8) AS PART8,
    EXPLT1.PLNT_ACRNYM,
    EXPLT1.PLNT_DESC,
    /** Identify duplicate values using row_number() function, partitioned by part8 and modified date **/
    row_number() OVER(PARTITION BY LEFT(EXWRK1.L1_DSG_CHG_PRT_NO,8), DATE(EXREP1.DATE_MODIFIED) ORDER BY EXWRK1.Q_CAL_FINAL_TOT_CS) AS ROW_NUM,
    EXWRK1.Q_CAL_FINAL_TOT_CS,
    EXQOT1.Q_EFF_START_DT,
    EXQOT1.Q_EFF_END_DT,
    DATE(EXREP1.DATE_MODIFIED) AS APPROVED_DATE

FROM
    EQX.EXWRK1 EXWRK1

    INNER JOIN EQX.EXQPL1 EXQPL1 ON
        EXWRK1.SUPLR_STG_ID = EXQPL1.SUPLR_STG_ID

    INNER JOIN EQX.EXPLT1 EXPLT1 ON
        EXPLT1.PLNT_ID = EXQPL1.PLNT_ID

    INNER JOIN EQX.EXREP1 EXREP1 ON
        EXWRK1.WRK_ID = EXREP1.WRK_ID

    INNER JOIN EQX.EXQOT1 EXQOT1 ON
        EXWRK1.SUPLR_STG_ID = EXQOT1.SUPLR_STG_ID

    LEFT JOIN EQX.EXSPP1 EXSPP1 ON
        EXWRK1.SUPLR_NO = EXSPP1.SUPLR_NO

    /** Code value can be used more than once based on COD_DOMAIN.  Only want the ones related to 'WORK_LIST_STATUS' **/
    LEFT JOIN (
        SELECT *

        FROM
            EQX.EXCOD1

        WHERE
            COD_DOMAIN = 'WORK_LIST_STATUS'
    ) EXCOD1 ON
        EXREP1.NEW_STATUS = EXCOD1.COD_VALUE

WHERE
    EXWRK1.SUPLR_NO = '518227'
    AND EXWRK1.L1_DSG_CHG_PRT_NO LIKE '200215%'
    AND EXCOD1.COD_NAME = 'Quote Approved MP'

ORDER BY
    PART8,
    DATE(EXREP1.DATE_MODIFIED),
    row_number() OVER(PARTITION BY LEFT(EXWRK1.L1_DSG_CHG_PRT_NO,8), DATE(EXREP1.DATE_MODIFIED) ORDER BY EXWRK1.Q_CAL_FINAL_TOT_CS)

) AS TEMP

WHERE
    ROW_NUM = 1
"""
try:
    print("Obtaining E-Quote records...")
    equote = pd.read_sql(sql, cnxn, index_col=None, parse_dates=['APPROVED_DATE'])
    cursor.close()
    cnxn.close()

except:
    # Close connections
    cursor.close()
    cnxn.close()
    print("Error occurred with connecting to the database")
print("Finished obtaining E-Quote mass production part costs")

Connecting to DSNOGW01 server
Obtaining E-Quote records...
Finished obtaining E-Quote mass production part costs


#### Define function to calculate mass pro cost at time of failure (RO date):

In [123]:
def getMassProCost(row):
    return equote[(equote['PART8'] == row['REPLACED_PART_EQUOTE_FMT']) & (equote['APPROVED_DATE'] ==
           equote[(equote['PART8'] == row['REPLACED_PART_EQUOTE_FMT']) & (equote['APPROVED_DATE'] <= row['REPAIR_ORDER_DATE'])]['APPROVED_DATE']
           .max())].sort_values(by=['Q_CAL_FINAL_TOT_CS'])['Q_CAL_FINAL_TOT_CS'].values[0]

#### Now, apply the function we defined above to create a new column containing the mass pro part cost at time of failure (RO Date):

In [124]:
print("Applying mass pro part cost x 205% calculations.  Please wait...")
claims_with_tpl['MassProCost_USD'] = claims_with_tpl.apply(getMassProCost, axis='columns')
claims_with_tpl = claims_with_tpl.assign(MassProCostX205_USD = claims_with_tpl['MassProCost_USD'] * 2.05)
print("Finished applying mass pro part cost x 205% calculations.")

Applying mass pro part cost x 205% calculations.  Please wait...
Finished applying mass pro part cost x 205% calculations.


In [125]:
claims_with_tpl.head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,WRPID,HRLY_RATE_USD,CALCULATED_PART_COST_USD,CALCULATED_FLAT_RATE_LABOR_COST_USD,CALCULATED_DIAGNOSTIC_LABOR_COST_USD,CALCULATED_HDLG_CHG_AMT_USD,CALCULATED_TOTAL_COST_USD,CALCULATED_TOTAL_LABOR_COST_USD,ROW_NUM,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD,ASN_RECEIVE_DATE,TransDTF,MassProCost_USD,MassProCostX205_USD
0,5FRYD4H85HB000032251369312271,201612,5FRYD4H85HB000032,5FRYD4H85HB000032,2016-12-23,2017-01-27,NaT,NaT,2017-01-27,CLOSED,CLOSED,6200,TRANSMISSION,35.00,35,062015NCA31,"CUSTOMER, STATES VEHICLE WILL NOT GO IN PARK A...",Visual inspection revealed ferrous debris in H...,Transmission shifted to Park after ignition wa...,HCU Contamination,HCU contamination caused claim of transmission...,251369,312271,"CUSTOMER, STATES VEHICLE WILL NOT GO IN PARK A...",2016-12-10,2016-12-19,2016-06-07,2016-06-06,218102,101,15A,KA,5.90,150.00,2.20,2016-12-21 10:28:30.350,100.00,Supplier Mfg. Defect,5267,$,"2,002.35","1,041.00","1,109.00","4,152.35",N,3214,Normal,1,6200,062005NCA31RM,WARRANTY A/T KIT,1,HTZ6,AJ6,,Q5NC0147177,ACURA,2017,HMA,MDX,9,200215NC,8.10,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",710960,1,WRP219B0918,141,"2,815.14",831.90,310.20,"1,970.60","6,077.84","1,142.10",1,"4,152.35",2016-06-03,190,"2,164.45","4,437.11"
1,5FRYD4H88HB000042251406314429,201803,5FRYD4H88HB000042,5FRYD4H88HB000042,2018-04-24,2018-06-07,2018-06-07,2018-06-26,2018-06-26,CLOSED,CLOSED,6200,TRANSMISSION,63.00,63,062275J4010,"ATF WARMER LEAKED AND, CAUSED TRANSMISSION TO ...",~ Defective oil cooler\r\n,~,Oil Cooler Diaphram Damaged,NaN,251406,314429,"ATF WARMER LEAKED AND, CAUSED TRANSMISSION TO ...",2018-03-16,2018-03-23,2016-06-07,2016-06-06,218102,101,15A,KA,5.90,0.00,2.20,2018-04-20 10:34:11.400,100.00,Supplier Mfg. Defect,18360,$,"2,891.96","1,015.99","2,445.23","6,353.18",N,8103,Normal,1,6200,062005NCA31RM ...,WARRANTY A/T KIT,1,HTZ6,AJ6,,Q5NC0147152,ACURA,2017,HMA,MDX,9,200215NC,8.10,1.00,"3,127.93","2,049.71","2,815.14","4,099.42","2,815.14",737496,1,WRP219B0918,141,"2,815.14",831.90,310.20,"1,970.60","5,927.84","1,142.10",1,"6,353.18",2016-06-03,651,"2,093.90","4,292.50"
2,5FRYD4H52HB000091251061931424,201705,5FRYD4H52HB000091,5FRYD4H52HB000091,2017-06-16,2017-08-07,2017-08-08,2017-09-20,2017-09-20,CLOSED,CLOSED,6200,TRANSMISSION,96.00,96,062005NCA21,CUSTOMER STATES TRANSMISSION IS SLIPPING,~ Completed visual inspection of transmission...,"~ During teardown drained oil, and found milk...",Oil Cooler Diaphram Damaged,Known issue related to oil cooler.,251061,931424,CUSTOMER STATES TRANSMISSION IS SLIPPING,2017-04-21,2017-05-09,2016-06-14,2016-06-13,218102,101,15A,KA,5.90,125.00,2.50,2017-06-14 11:12:08.720,100.00,Supplier Mfg. Defect,5533,$,"3,024.98","1,259.00","2,223.01","6,506.99",N,2301,Normal,1,6200,062005NCA21RM,WARRANTY A/T KIT

In [126]:
claims_with_tpl[['REPLACED_PART_EQUOTE_FMT','REPAIR_ORDER_DATE','MassProCost_USD','MassProCostX205_USD']].head()

,REPLACED_PART_EQUOTE_FMT,REPAIR_ORDER_DATE,MassProCost_USD,MassProCostX205_USD
0,200215NC,2016-12-10,"2,164.45","4,437.11"
1,200215NC,2018-03-16,"2,093.90","4,292.50"
2,200215NC,2017-04-21,"2,173.67","4,456.02"
3,200215NC,2017-05-20,"2,173.68","4,456.04"
4,200215NC,2018-05-11,"2,070.46","4,244.45"


### 'zero out' cost amounts for duplicate rows to prevent double-charging of supplier

In [127]:
claims_with_tpl['ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD'] = np.where(claims_with_tpl['ROW_NUM'] == 1,
                                                                  claims_with_tpl['ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD'], 0)
claims_with_tpl['FLAT_RATE_HRS_QTY'] = np.where(claims_with_tpl['ROW_NUM'] == 1,
                                               claims_with_tpl['FLAT_RATE_HRS_QTY'], 0)
claims_with_tpl['CALCULATED_PART_COST_USD'] = np.where(claims_with_tpl['ROW_NUM'] == 1,
                                                      claims_with_tpl['CALCULATED_PART_COST_USD'], 0)
claims_with_tpl['CALCULATED_FLAT_RATE_LABOR_COST_USD'] = np.where(claims_with_tpl['ROW_NUM'] == 1,
                                                                 claims_with_tpl['CALCULATED_FLAT_RATE_LABOR_COST_USD'], 0)
claims_with_tpl['CALCULATED_DIAGNOSTIC_LABOR_COST_USD'] = np.where(claims_with_tpl['ROW_NUM'] == 1,
                                                                  claims_with_tpl['CALCULATED_DIAGNOSTIC_LABOR_COST_USD'], 0)
claims_with_tpl['CALCULATED_HDLG_CHG_AMT_USD'] = np.where(claims_with_tpl['ROW_NUM'] == 1,
                                                         claims_with_tpl['CALCULATED_HDLG_CHG_AMT_USD'], 0)
claims_with_tpl['CALCULATED_TOTAL_LABOR_COST_USD'] = np.where(claims_with_tpl['ROW_NUM'] == 1,
                                                             claims_with_tpl['CALCULATED_TOTAL_LABOR_COST_USD'], 0)
claims_with_tpl['CALCULATED_TOTAL_COST_USD'] = np.where(claims_with_tpl['ROW_NUM'] == 1,
                                                       claims_with_tpl['CALCULATED_TOTAL_COST_USD'], 0)
claims_with_tpl['DIAGNOSTIC_LABOR_HRS_QTY'] = np.where(claims_with_tpl['ROW_NUM'] == 1,
                                                      claims_with_tpl['DIAGNOSTIC_LABOR_HRS_QTY'], 0)
claims_with_tpl['FRGT_SUBL_TAX_AMT_USD'] = np.where(claims_with_tpl['ROW_NUM'] == 1,
                                                   claims_with_tpl['FRGT_SUBL_TAX_AMT_USD'], 0)
claims_with_tpl['MassProCost_USD'] = np.where(claims_with_tpl['ROW_NUM'] == 1,
                                             claims_with_tpl['MassProCost_USD'], 0)
claims_with_tpl['MassProCostX205_USD'] = np.where(claims_with_tpl['ROW_NUM'] == 1,
                                                 claims_with_tpl['MassProCostX205_USD'], 0)

In [128]:
duplicate_rows = claims_with_tpl[claims_with_tpl.duplicated(subset=['VIN_DLR_CLM_SK'], keep=False)]
duplicate_rows.sort_values(['VIN_DLR_CLM_SK', 'ROW_NUM'])[:5]

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,WRPID,HRLY_RATE_USD,CALCULATED_PART_COST_USD,CALCULATED_FLAT_RATE_LABOR_COST_USD,CALCULATED_DIAGNOSTIC_LABOR_COST_USD,CALCULATED_HDLG_CHG_AMT_USD,CALCULATED_TOTAL_COST_USD,CALCULATED_TOTAL_LABOR_COST_USD,ROW_NUM,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD,ASN_RECEIVE_DATE,TransDTF,MassProCost_USD,MassProCostX205_USD
1359,19UUB2F52HA003984251551361711,201807,19UUB2F52HA003984,19UUB2F52HA003984,2018-07-17,2018-08-29,2018-09-06,2018-09-07,2018-09-07,CLOSED,CLOSED,6200,TRANSMISSION,52.00,52,062015J4A73,CUSTOMER STATES TRANSMISSION SYSTEM PROBLEM LI...,~ Visual Inspection of transmission revealed ...,~ Full teardown performed and all clutches me...,NTF,HCU and sensor unit were sent for functional t...,251551,361711,CUSTOMER STATES TRANSMISSION SYSTEM PROBLEM LI...,2018-06-26,2018-07-05,2016-12-07,2016-12-06,218102,101,101,KA,5.80,125.00,1.40,2018-07-13 08:27:44.780,0.00,NTF,24298,$,"2,769.59","1,025.00","1,107.84","4,902.43",N,3214,Normal,1,6200,062005J4A71RM ...,WARRANTY A/T KIT,1,HTZ3,AB9,,Q5J40154391,ACURA,2017,MAP,TLX,9,200215J4,7.20,1.00,"2,967.12","2,433.04","2,670.41","4,866.08","2,670.41",740894,1,WRP219B0918,141,"2,670.41",817.80,197.40,"1,869.29","5,679.90","1,015.20",1,969.36,2016-11-28,575,"1,951.03","3,999.62"
1358,19UUB2F52HA003984251551361711,201902,19UUB2F52HA003984,19UUB2F52HA003984,2018-07-17,2018-08-29,2018-09-06,2018-09-07,2018-09-07,CLOSED,CLOSED,6200,TRANSMISSION,52.00,52,062015J4A73,CUSTOMER STATES TRANSMISSION SYSTEM PROBLEM LI...,~ Visual Inspection of transmission revealed ...,~ Full teardown performed and all clutches me...,NTF,HCU and sensor unit were sent for functional t...,251551,361711,CUSTOMER STATES TRANSMISSION SYSTEM PROBLEM LI...,2018-06-26,2018-07-05,2016-12-07,2016-12-06,218102,101,101,KA,0.00,0.00,0.00,2018-07-13 08:27:44.780,0.00,NTF,24298,$,"-1,800.23","-1,025.00","-1,107.84","-3,933.07",N,99900,Debit,-1,6200,062005J4A71RM ...,WARRANTY A/T KIT,1,HTZ3,AB9,,Q5J40154391,ACURA,2017,MAP,TLX,9,200215J4,0.00,1.00,"2,967.12","2,433.04","2,670.41","4,866.08","2,670.41",740894,1,WRP219B0918,141,0.00,0.00,0.00,0.00,0.00,0.00,2,0.00,2016-11-28,575,0.00,0.00
1303,19UUB2F57HA001020251024381631,201811,19UUB2F57HA001020,19UUB2F57HA001020,2018-12-17,2019-02-07,2019-03-25,NaT,NaT,REOPENED_FOR_ANALYSIS,OPEN,6200,TRANSMISSION,nan,127,062005J4A71RM,MISC REPAIR CUSTOMER STATES THAT TRANSMISSION ...,~,~,NaN,NaN,251024,381631,MISC REPAIR CUSTOMER STATES THAT TRANSMISSION ...,2018-10-16,2018-11-16,2016-08-26,2016-08-24,218102,101,101,KA,5.80,250.00,1.40,2018-12-13 09:49:05.587,nan,NaN,30963,$,"2,710.89","1,257.64","1,084.35","5,052.88",N,3214,Normal,1,6200,062005J4A71RM ...,WARRANTY A/T KIT,1,HTZ3,AB9,,Q5J40136178,ACURA,2017,MAP,TLX,9,200215J4,7.20,1.00,"2,967.12","2,433.04","2,670.41","4,866.08","2,670.41",746531,1,WRP219B

### Add total and calculated total cost amounts times responsibility percentage:

#### But first, must fill missing responsibility percentage with 0/zero:

In [129]:
claims_with_tpl['RESPONSIBILITY_PCT'].fillna(value=0, inplace=True)

In [130]:
claims_with_tpl = claims_with_tpl.assign(ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC =
                               claims_with_tpl['ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD'] * (claims_with_tpl['RESPONSIBILITY_PCT'] / 100))

In [131]:
claims_with_tpl = claims_with_tpl.assign(CALCULATED_TOTAL_COST_USD_AFTER_PERC =
                               claims_with_tpl['CALCULATED_TOTAL_COST_USD'] * (claims_with_tpl['RESPONSIBILITY_PCT'] / 100))

In [132]:
claims_with_tpl[['RESPONSIBILITY_PCT','ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD',
                 'ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC','CALCULATED_TOTAL_COST_USD','CALCULATED_TOTAL_COST_USD_AFTER_PERC']].head()

,RESPONSIBILITY_PCT,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC,CALCULATED_TOTAL_COST_USD,CALCULATED_TOTAL_COST_USD_AFTER_PERC
0,100.00,"4,152.35","4,152.35","6,077.84","6,077.84"
1,100.00,"6,353.18","6,353.18","5,927.84","5,927.84"
2,100.00,"6,506.99","6,506.99","6,095.14","6,095.14"
3,100.00,"6,448.38","6,448.38","5,911.84","5,911.84"
4,0.00,"21,354.19",0.00,"7,617.94",0.00


### Add initial and final WRP amount columns in USD: min(actual total cost, calculated cost, MassPro205 cost)

In [133]:
claims_with_tpl = claims_with_tpl.assign(WRP_VOUCHER_AMT_USD_INITIAL = claims_with_tpl[['ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC',
                                                                       'CALCULATED_TOTAL_COST_USD_AFTER_PERC',
                                                                       'MassProCostX205_USD']].min(axis='columns'))

In [134]:
claims_with_tpl[['ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC','CALCULATED_TOTAL_COST_USD_AFTER_PERC','MassProCostX205_USD',
                'WRP_VOUCHER_AMT_USD_INITIAL']].head()

,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC,CALCULATED_TOTAL_COST_USD_AFTER_PERC,MassProCostX205_USD,WRP_VOUCHER_AMT_USD_INITIAL
0,"4,152.35","6,077.84","4,437.11","4,152.35"
1,"6,353.18","5,927.84","4,292.50","4,292.50"
2,"6,506.99","6,095.14","4,456.02","4,456.02"
3,"6,448.38","5,911.84","4,456.04","4,456.04"
4,0.00,0.00,"4,244.45",0.00


In [135]:
claims_with_tpl = claims_with_tpl.assign(WRP_VOUCHER_AMT_USD_FINAL=claims_with_tpl['WRP_VOUCHER_AMT_USD_INITIAL'])

### Create initial voucher based on the following criteria:

```
STATUS_NAME IN('CLOSED','VOUCHERED')
and (
    (SUBMITTED_DATE >= voucher start date and SUBMITTED_DATE <= voucher end date
     and NOT REOPENED_SUBMITTED_DATE > voucher end date
    )
    or (
        REOPENED_SUBMITTED_DATE >= voucher start date and REOPENED_SUBMITTED_DATE <= voucher end date
    )
)
and RESPONSIBILITY_PCT > 0
and ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC > 0.1
and vehicle miles <= 70000
and TransDTF <= 1460 (4 years)
and CAMPAIGN_CODE = 'N'
and NOT CONCLUSION_DESC_TXT = 'NO REPLY'```

In [136]:
%%script false
criteria1 = claims_with_tpl['STATUS_NAME'].isin(['CLOSED','VOUCHERED'])
criteria2 = (claims_with_tpl['SUBMITTED_DATE'] >= start_voucher_date) \
                                    & (claims_with_tpl['SUBMITTED_DATE'] <= end_voucher_date)
criteria2a = claims_with_tpl['REOPENED_SUBMITTED_DATE'] > end_voucher_date
criteria3 = (claims_with_tpl['REOPENED_SUBMITTED_DATE'] >= start_voucher_date) \
                                    & (claims_with_tpl['REOPENED_SUBMITTED_DATE'] <= end_voucher_date)
criteria4 = claims_with_tpl['RESPONSIBILITY_PCT'] > 0
criteria5 = claims_with_tpl['ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD'] > 0.1
criteria6 = claims_with_tpl['VEH_MILEAGE'] <= 70000
criteria7 = claims_with_tpl['TransDTF'] <= 1460
criteria8 = claims_with_tpl['CAMPAIGN_CODE'] == 'N'
criteria9 = claims_with_tpl['CONCLUSION_DESC_TXT'] == 'NO REPLY'

In [137]:
%%script false
voucher_nonfixed = claims_with_tpl[criteria1
                                  & ((criteria2 & ~criteria2a) | criteria3)
                                  & criteria4
                                  & criteria5
                                  & criteria6
                                  & criteria7
                                  & criteria8
                                  & ~criteria9]

In [138]:
criteria1 = claims_with_tpl['STATUS_NAME'].isin(['CLOSED','VOUCHERED'])
criteria2 = (claims_with_tpl['FINAL_CLOSURE_DATE'] >= start_voucher_date) \
                                    & (claims_with_tpl['FINAL_CLOSURE_DATE'] <= end_voucher_date)
criteria3 = claims_with_tpl['RESPONSIBILITY_PCT'] > 0
criteria4 = claims_with_tpl['ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD'] > 0.1
criteria5 = claims_with_tpl['VEH_MILEAGE'] <= 70000
criteria6 = claims_with_tpl['TransDTF'] <= 1460
criteria7 = claims_with_tpl['CAMPAIGN_CODE'] == 'N'
criteria8 = claims_with_tpl['CONCLUSION_DESC_TXT'] == 'NO REPLY'

In [139]:
voucher_nonfixed = claims_with_tpl[criteria1
                                  & criteria2
                                  & criteria3
                                  & criteria4
                                  & criteria5
                                  & criteria6
                                  & criteria7
                                  & ~criteria8]

In [140]:
claims_with_tpl.shape

(1479, 89)

In [141]:
voucher_nonfixed.shape

(92, 89)

#### Create FIXED_YES_NO and SUPP_FIXED_PERC columns

In [142]:
voucher_nonfixed = voucher_nonfixed.assign(FIXED_YES_NO = 'NO')
voucher_nonfixed = voucher_nonfixed.assign(SUPP_FIXED_PERC = 'Not Applicable')

### Obtain already paid claims to prevent double-charging of supplier:

In [143]:
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=\\mmpapp02\mq_db\wrp\ZF_ATM_WRP_Project\version4\databases\WRP_DAT_ZF.accdb;'
)
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

sql = """
SELECT
    *

FROM
    tbl_Already_Paid_Claims
"""

try:
    already_paid_claims = pd.read_sql(sql, cnxn)

    # Close connections
    cursor.close()
    cnxn.close()
    print(sql)
except:
    print("Error connecting to database")
    cursor.close()
    cnxn.close()


SELECT
    *

FROM
    tbl_Already_Paid_Claims



In [144]:
voucher_nonfixed_final = pd.merge(voucher_nonfixed, already_paid_claims, how='left', left_on='VIN_DLR_CLM_SK',
                                    right_on='VIN_DLR_CLM_SK', indicator=True)

In [145]:
voucher_nonfixed_final.shape

(92, 94)

In [146]:
voucher_nonfixed_final.head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,WRPID,HRLY_RATE_USD,CALCULATED_PART_COST_USD,CALCULATED_FLAT_RATE_LABOR_COST_USD,CALCULATED_DIAGNOSTIC_LABOR_COST_USD,CALCULATED_HDLG_CHG_AMT_USD,CALCULATED_TOTAL_COST_USD,CALCULATED_TOTAL_LABOR_COST_USD,ROW_NUM,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD,ASN_RECEIVE_DATE,TransDTF,MassProCost_USD,MassProCostX205_USD,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC,CALCULATED_TOTAL_COST_USD_AFTER_PERC,WRP_VOUCHER_AMT_USD_INITIAL,WRP_VOUCHER_AMT_USD_FINAL,FIXED_YES_NO,SUPP_FIXED_PERC,WRP_ID,WRP_RECOVERY_COST_AFTER_PERC_USD_FINAL,_merge
0,5J8YD4H54HL003171251241349358,201806,5J8YD4H54HL003171,5J8YD4H54HL003171,2018-06-22,2018-08-13,2018-08-13,2018-09-06,2018-09-06,CLOSED,CLOSED,6233,KIT,76.00,76,200215L9053,NaN,~ End cover gasket found to be leaking,~,EK Gasket Leak,NaN,251241,349358,NaN,2018-06-05,2018-06-12,2017-08-04,2017-08-01,2321S2,101,160,KA,1.00,0.00,0.00,2018-06-20 09:58:02.603,100.00,Supplier Mfg. Defect,3966,$,26.06,159.00,16.04,201.10,N,7408,Normal,1,6233,062335J4000 ...,COVER KIT,1,HTYS,AG6,,Q5NC0353213,ACURA,2017,ELP,MDX,9,200215NC,1.00,1.00,40.93,12.02,36.84,24.04,24.04,740033,1,WRP219B0918,141,24.04,141.00,0.00,25.79,190.83,141.00,1,201.10,2017-07-26,314,"2,070.46","4,244.45",201.10,190.83,190.83,190.83,NO,Not Applicable,NaN,nan,left_only
1,5J8YD4H32HL007721251069756668,201808,5J8YD4H32HL007721,5J8YD4H32HL007721,2018-09-06,2018-09-14,NaT,NaT,2018-09-14,CLOSED,CLOSED,6233,KIT,8.00,8,200215NB043,CUSTOMER STATES TRANSMISSION FLUID LEAKING CHE...,End cover gasket found defective ~,~,EK Gasket Leak,NaN,251069,756668,CUSTOMER STATES TRANSMISSION FLUID LEAKING CHE...,2018-08-15,2018-08-16,2017-09-25,2017-09-20,2321S2,101,160,KA,1.00,0.00,0.00,2018-09-04 12:08:32.640,100.00,Supplier Mfg. Defect,381,$,102.94,130.00,98.81,331.75,N,7408,Normal,1,6233,062335J4000 ...,COVER KIT,1,HTYS,AA8,,Q5NC0379516,ACURA,2017,ELP,MDX,9,200215NC,1.00,1.00,40.93,12.02,36.84,24.04,24.04,742937,1,WRP219B0918,141,24.04,141.00,0.00,25.79,190.83,141.00,1,331.75,2017-09-11,338,"2,078.91","4,261.76",331.75,190.83,190.83,190.83,NO,Not Applicable,NaN,nan,left_only
2,5J8YD4H35JL003104251554059141,201806,5J8YD4H35JL003104,5J8YD4H35JL003104,2018-06-18,2018-08-07,2018-08-07,2018-09-06,2018-09-06,CLOSED,CLOSED,6233,KIT,80.00,80,062335J4000,DIAGNOSIS (TRANSMISSION - NOT SHIFTING PROPERL...,~ End cover gasket found to be leaky,~,EK Gasket Leak,NaN,251554,059141,DIAGNOSIS (TRANSMISSION - NOT SHIFTING PROPERL...,2018-05-25,2018-05-29,2017-11-02,2017-11-01,2181A4,101,160,KA,0.70,0.00,0.00,2018-06-14 10:44:50.223,100.00,Supplier Mfg. Defect,14640,$,101.74,87.48,108.71,297.93,N,1801,Normal,1,6233,062335J4000 ...,COVER KIT,1,JTYS,AA8,,Q5DH0403317,ACURA,2018,ELP,MDX,9,200215DH,0.70,1.00,40.93,12.02,36.84,24.04,24.04,739715,1,WRP219B0918,141,24.04,98.70,0.00,25.79,148.53,98.7

In [147]:
voucher_nonfixed_final['_merge'].value_counts()

left_only     92
both           0
right_only     0
Name: _merge, dtype: int64

### Exclude already paid claims to prevent double-charging of supplier:

In [148]:
criteria = voucher_nonfixed_final['_merge'] == 'left_only'
voucher_nonfixed_final = voucher_nonfixed_final[criteria]

In [149]:
voucher_nonfixed_final.shape

(92, 94)

### Delete redundant or unnecessary columns

In [150]:
del voucher_nonfixed_final['_merge']
del voucher_nonfixed_final['WRP_ID']
del voucher_nonfixed_final['WRP_RECOVERY_COST_AFTER_PERC_USD_FINAL']

### Save voucher data as Excel file:

In [151]:
today = datetime.today()

In [152]:
base_dir = "//mmpapp02/mq_db/wrp/ZF_ATM_WRP_Project/automated_vouchers/outputs/excel/NorthAmericanPlants/NonFixed"
p = Path(base_dir)
save_dir = p / datetime.strftime(today, "%Y-%m-%d")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [153]:
file_name = 'NA_Plants_NonFixed_Voucher_FINAL.xlsx'

In [154]:
voucher_nonfixed_final.to_excel(save_dir / file_name, index=False)

#### Where Excel file was saved to:

In [155]:
str(save_dir / file_name)

'\\\\mmpapp02\\mq_db\\wrp\\ZF_ATM_WRP_Project\\automated_vouchers\\outputs\\excel\\NorthAmericanPlants\\NonFixed\\2019-04-23\\NA_Plants_NonFixed_Voucher_FINAL.xlsx'

### Send out Windows notification when complete

In [156]:
toaster = ToastNotifier()
toaster.show_toast("Papermill Orchestration Status",
                   "Successfully created non-fixed voucher",
                   icon_path="images/honda_logo.ico",
                   duration=5)

True